# Data Preprocessing and testing with VecDb

In [12]:
import pandas as pd

In [13]:
df=pd.read_excel("Data.xlsx")
df.columns={"Data"}
df

,Data
0,1. Andhra Pradesh: Capital - Amravati. Known f...
1,2. Arunachal Pradesh: Capital - Itanagar. Famo...
2,3. Assam: Capital - Dispur. Renowned for its t...
3,4. Bihar: Capital - Patna. Historically signif...
4,5. Chhattisgarh: Capital - Raipur. Known for i...
5,6. Goa: Capital - Panaji. Famous for its beaut...
6,7. Gujarat: Capital - Gandhinagar. Known for i...
7,8. Haryana: Capital - Chandigarh. An agricultu...
8,9. Himachal Pradesh: Capital - Shimla. A parad...
9,10. Jharkhand: Capital - Ranchi. Rich in miner...


In [15]:
Data = []
for i in df["Data"]:
    Data.append(str(i))

In [16]:
type(Data[0])

str

In [17]:
ids =[str(i+1) for i in range(len(Data))]

In [18]:
import chromadb
chroma_client = chromadb.Client()

In [19]:
collection = chroma_client.create_collection(name="my_collection")

UniqueConstraintError: Collection my_collection already exists

In [20]:
collection.add(
    documents=Data,ids=ids)

In [38]:
results = collection.query(
    query_texts="what is the capital of kerala",
    n_results=2
)
results

{'ids': [['12', '11']],
 'distances': [[0.5875943303108215, 1.0149141550064087]],
 'metadatas': [[None, None]],
 'embeddings': None,
 'documents': [['12. Kerala: Capital - Thiruvananthapuram. Famous for its backwaters, Ayurvedic treatments, and lush green landscapes.',
   '11. Karnataka: Capital - Bengaluru. The IT capital of India, with a blend of modernity and tradition.']],
 'uris': None,
 'data': None}

# Integration of LLM

In [22]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load tokenizer and model
model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)



In [29]:
import os
from gtts import gTTS
# Define input text
question =results.text
input_text = f"""
    Based on the below data,
    {question}.
    {collection.query(
    query_texts=f"{question}",
    n_results=1)}
"""

# Tokenize input text
inputs = tokenizer(input_text, max_length=1024, return_tensors="pt", truncation=True)

# Generate summary
summary_ids = model.generate(inputs["input_ids"], max_length=150, min_length=40, num_beams=4, early_stopping=True)

# Decode and print the summary
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
#print("Generated Summary:", summary)
myobj = gTTS(text=summary, lang="en", slow=False)
myobj.save("welcome.mp3")
# Play the converted file
os.system("welcome.mp3")


0

# recognising the user audio input

In [27]:
import whisper
model1 = whisper.load_model("base.en")

In [134]:
def detect_audio_start():
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 16000  # Sample rate in Hz
    RECORD_SECONDS = 10  # Maximum recording time

    p = pyaudio.PyAudio()

    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

In [39]:
import pyaudio
import wave
#import webrtcvad

def record_audio(output_file, sample_rate=16000, duration=5):
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 1

    p = pyaudio.PyAudio()

    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=sample_rate,
                    input=True,
                    frames_per_buffer=CHUNK)

    print("Recording...")
    frames = []

    # Record audio
    for _ in range(0, int(sample_rate / CHUNK * duration)):
        audio_data = stream.read(CHUNK)
        frames.append(audio_data)

        '''if is_speech(audio_data, sample_rate):
            print("Speech detected.")'''

    print("Finished recording.")

    # Stop stream
    stream.stop_stream()
    stream.close()
    p.terminate()

    # Save audio to file
    wf = wave.open(output_file, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(sample_rate)
    wf.writeframes(b''.join(frames))
    wf.close()

if __name__ == "__main__":
    output_file = "recorded_audio.wav"
    record_audio(output_file)
    print("Audio saved as", output_file)
 

Recording...
Finished recording.
Audio saved as recorded_audio.wav


In [24]:
import torchaudio
import whisper
waveform, sample_rate = torchaudio.load("recorded_audio.wav")
audio = whisper.pad_or_trim(waveform.flatten()).to("cuda")
mel = whisper.log_mel_spectrogram(audio)

In [25]:
options = whisper.DecodingOptions( without_timestamps=True,language="en")

In [28]:
results=model1.decode(mel,options)
results.text

'What is the capital of Andhra Pradesh?'

In [32]:
collections =chroma_client.list_collections()
collections

[Collection(name=my_collection)]

In [35]:
chroma_client.get_collection("my_collection") in collections 

False

In [34]:
chroma_client.get_collection("my_collection")

Collection(name=my_collection)